# The Power of Concurrency in Ollama: A Timed Comparison

This notebook will practically demonstrate the speed and efficiency benefits of concurrent model serving in Ollama.


We will time this entire workflow in two scenarios:
* **Scenario A (Sequential):** Forcing a "cold start" for each step by loading and unloading each model.
* **Scenario B (Concurrent):** Pre-loading all models into VRAM to make them "hot" and instantly available.

In [1]:
!pip install ollama
import ollama
import time
import os

# --- Configuration ---
# Make sure you have these models pulled!
# Run this in your terminal if you don't:

Model1 = 'gemma2:2b'
EMBED_MODEL = 'nomic-embed-text'
USER_PROMPT = "Write a short Python script to list all files in a directory."

## Scenario A: "Sequential" Model

In [5]:
start_time_a = time.time()

try:
    # --- Step 1}: Classify (Cold Start) ---
    print(f"Step 1: Classifying (Loading {Model1})...")
    step1_start = time.time()
    classify_response = ollama.chat(
        model=Model1,
        messages=[{
            'role': 'system', 
            'content': "Classify the user's intent as 'coding', 'billing', or 'general'. Respond with one word."
        }, {
            'role': 'user', 
            'content': USER_PROMPT
        }] 
    )
    print(f"Step 1 Complete ({(time.time() - step1_start):.2f}s). Intent: {classify_response['message']['content']}")

    # --- Step 2: Embed (Cold Start) ---
    print(f"\nStep 2: Embedding (Loading {EMBED_MODEL})...")
    step2_start = time.time()
    embed_response = ollama.embeddings(
        model=EMBED_MODEL,
        prompt=USER_PROMPT
    )
    print(f"Step 2 Complete ({(time.time() - step2_start):.2f}s). Embedding dim: {len(embed_response['embedding'])}")

    # --- Step 3: Generate (Cold Start) ---
    print(f"\nStep 3: Generating (Loading {Model1})...")
    step3_start = time.time()
    generate_response = ollama.chat(
        model=Model1,
        messages=[{'role': 'user', 'content': f"Request: {USER_PROMPT}"}],
    )
    print(f"Step 3 Complete ({(time.time() - step3_start):.2f}s).")
    # print(f"Response: {generate_response['message']['content'][:50]}...")

except ollama.ResponseError as e:
    print(f"Error: {e.error}")
    print("Please make sure you have pulled all three models: 'mistral:7b', 'nomic-embed-text', and 'llama3:8b'")

end_time_a = time.time()
total_time_a = end_time_a - start_time_a

print("-" * 30)
print(f"--- SCENARIO A TOTAL TIME: {total_time_a:.2f} seconds ---")

Step 1: Classifying (Loading gemma2:2b)...
Step 1 Complete (3.72s). Intent: ```python
import os

def list_files(directory):
  """Lists all files in the specified directory."""
  for filename in os.listdir(directory):
    print(filename)

# Example usage:
list_files("/path/to/your/directory") 
``` 

**Intent:** **coding** 


Step 2: Embedding (Loading nomic-embed-text)...
Step 2 Complete (1.20s). Embedding dim: 768

Step 3: Generating (Loading gemma2:2b)...
Step 3 Complete (5.75s).
------------------------------
--- SCENARIO A TOTAL TIME: 10.67 seconds ---


In [3]:
start_time_b = time.time()

# --- Step 1: Classify (Hot Run) ---
print(f"Step 1: Classifying (Hot Run)...")
step1_start = time.time()
classify_response = ollama.chat(
    model=Model1,
    messages=[{
        'role': 'system', 
        'content': "Classify the user's intent as 'coding', 'billing', or 'general'. Respond with one word."
    }, {
        'role': 'user', 
        'content': USER_PROMPT
    }]
)
print(f"Step 1 Complete ({(time.time() - step1_start):.2f}s). Intent: {classify_response['message']['content']}")

# --- Step 2: Embed (Hot Run) ---
print(f"\nStep 2: Embedding (Hot Run)...")
step2_start = time.time()
embed_response = ollama.embeddings(
    model=EMBED_MODEL,
    prompt=USER_PROMPT
)
print(f"Step 2 Complete ({(time.time() - step2_start):.2f}s). Embedding dim: {len(embed_response['embedding'])}")

# --- Step 3: Generate (Hot Run) ---
print(f"\nStep 3: Generating (Hot Run)...")
step3_start = time.time()
generate_response = ollama.chat(
    model=Model1,
    messages=[{'role': 'user', 'content': f"Request: {USER_PROMPT}"}]
)
print(f"Step 3 Complete ({(time.time() - step3_start):.2f}s).")


end_time_b = time.time()
total_time_b = end_time_b - start_time_b

print("-" * 30)
print(f"--- SCENARIO B TOTAL TIME: {total_time_b:.2f} seconds ---")

Step 1: Classifying (Hot Run)...
Step 1 Complete (1.55s). Intent: ```python
import os

def list_files(directory):
  """Lists all files in the specified directory."""
  for filename in os.listdir(directory):
    print(filename) 

# Example usage:
list_files('/path/to/your/directory') 
```


**Classification:** **coding** 


Step 2: Embedding (Hot Run)...
Step 2 Complete (0.06s). Embedding dim: 768

Step 3: Generating (Hot Run)...
Step 3 Complete (5.19s).
------------------------------
--- SCENARIO B TOTAL TIME: 6.80 seconds ---


In [4]:
print("--- FINAL ANALYSIS ---")
print(f"Scenario A (Sequential Cold Starts) Time: {total_time_a:.2f} seconds")
print(f"Scenario B (Concurrent Hot Run) Time:    {total_time_b:.2f} seconds")
print("-" * 30)

if total_time_b > 0:
    difference = total_time_a - total_time_b
    performance_gain = (total_time_a / total_time_b)
    print(f"Concurrency saved {difference:.2f} seconds.")
    print(f"The concurrent workflow was {performance_gain:.1f}x faster.")
else:
    print("Scenario B was too fast to measure or an error occurred.")

--- FINAL ANALYSIS ---
Scenario A (Sequential Cold Starts) Time: 13.74 seconds
Scenario B (Concurrent Hot Run) Time:    6.80 seconds
------------------------------
Concurrency saved 6.94 seconds.
The concurrent workflow was 2.0x faster.


In [ ]:
# concurrency depends on Memory

# EXAMPLE -2

In [6]:
start_time_1 = time.time()

output = ollama.chat(
    model='mistral:7b',
    messages=[{
        'role': 'system', 
        'content': "Classify the user's intent as 'coding', 'billing', or 'general'. Respond with one word."
    }, {
        'role': 'user', 
        'content': "What is photosynthesis"
    }]
)

print(output['message']['content'])

end_time_1= time.time()

total_time_1 = end_time_1- start_time_1
print(total_time_1)

 general
11.206955671310425


In [7]:
start_time_2 = time.time()

output = ollama.chat(
    model='mistral:7b',
    messages=[{
        'role': 'system', 
        'content': "Classify the user's intent as 'coding', 'billing', or 'general'. Respond with one word."
    }, {
        'role': 'user', 
        'content': "What is GenAI"
    }]
)

print(output['message']['content'])

end_time_2= time.time()

total_time_2 = end_time_2- start_time_2
print(total_time_2)

 Coding
0.481203556060791


In [9]:
print("--- FINAL ANALYSIS ---")
print(f"Scenario A (Sequential Cold Starts) Time: {total_time_1:.2f} seconds")
print(f"Scenario B (Concurrent Hot Run) Time:    {total_time_2:.2f} seconds")
print("-" * 30)

if total_time_2 > 0:
    difference = total_time_1 - total_time_2
    performance_gain = (total_time_1 / total_time_2)
    print(f"Concurrency saved {difference:.2f} seconds.")
    print(f"The concurrent workflow was {performance_gain:.1f}x faster.")
else:
    print("Scenario B was too fast to measure or an error occurred.")

--- FINAL ANALYSIS ---
Scenario A (Sequential Cold Starts) Time: 11.21 seconds
Scenario B (Concurrent Hot Run) Time:    0.48 seconds
------------------------------
Concurrency saved 10.73 seconds.
The concurrent workflow was 23.3x faster.


In [19]:
# previous models remvoed becasuse as per my working memory only 4gb model can be on my speed disk and other will be removed...